In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # Not used right now

import trackpy as tp

%matplotlib notebook

# Optionally, tweak styles.
mpl.rc('figure',  figsize=(8, 8))
mpl.rc('image', cmap='gray')


In [ ]:
import cv2
import numpy as np

file_name = "Pool.mp4"
#file_name = "LampVid2.mp4"
cap = cv2.VideoCapture(file_name)

imgs = []
cimgs = []
mimgs = []

ret, img = cap.read()
r = cv2.selectROI('Select your ROI with the mouse.  Press Enter/Space to finalize',img)
# Space / Enter to confirm selection
#cv2.waitKey(0)
cv2.destroyWindow('Select your ROI with the mouse.  Press Enter/Space to finalize')
img = img[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]

imgs.append(img)

#change range to load more frames
for x in range(100):
    ret, img = cap.read()
    img = img[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
    imgs.append(img)

'''
cimgs.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
img_b,img_g,img_r = cv2.split(img)
mimgs.append(cv2.addWeighted(img_r,.8,img_b,.8,10))

for x in range(100):
    ret, img = cap.read()
    img = img[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
    imgs.append(img)
    cimgs.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    img_b,img_g,img_r = cv2.split(img)
    mimgs.append(cv2.addWeighted(img_r,.8,img_b,.8,10))
'''

fig,ax = plt.subplots()
ax.imshow(imgs[0])
plt.show()

print("ROI selected")

In [ ]:
avg_img = np.mean(imgs[:], axis=0).astype(np.uint8)   #Averages all imgs to get background
bkg_img = cv2.blur(avg_img,(2,2)) #blurs the averaged image using a 25x25 square on each pixel.  
#Helps ensure that the particles themselves are less likely to be counted in the background

diff_video = []
display_video = []

# The following code is for gamma correction for humans, since it can be really hard to see the particles after background subtraction.
gamma = 0.2
inv_gamma = 1.0 / gamma
table = np.array([((i / 255.0) ** inv_gamma) * 255
    for i in np.arange(0, 256)]).astype("uint8")

for item in imgs:
    diff_video.append(cv2.bitwise_not(cv2.absdiff(item, bkg_img)))
    display_video.append(cv2.LUT(cv2.bitwise_not(cv2.absdiff(item, bkg_img)), table))
print("Background Generated")

lazy_subtraction = cv2.bitwise_not(cv2.absdiff(imgs[0], avg_img))

In [ ]:
import ipywidgets as widgets
from ipywidgets import Button, GridBox, Layout, ButtonStyle

up_control = widgets.Button(description="🠉",layout=Layout(width='auto',grid_area='up'))
down_control = widgets.Button(description='🠋',layout=Layout(width='auto',grid_area='down'))
left_control = widgets.Button(description='🠈',layout=Layout(width='auto',grid_area='left'))
right_control = widgets.Button(description='🠊',layout=Layout(width='auto',grid_area='right'))
deselect = widgets.Button(description='⊘',layout=Layout(width='auto',grid_area='deselect'))
next_frame = widgets.Button(description='⇒',layout=Layout(width='auto',grid_area='n_frame'))
prev_frame = widgets.Button(description='⇐',layout=Layout(width='auto',grid_area='p_frame'))


arrow_control = GridBox(children = [up_control,down_control,left_control,right_control,deselect,next_frame,prev_frame],
       layout=Layout(
       width='100px',
       height='200px',
       grid_template_rows = "100px auto auto auto",
       grid_template_columns = "33% 33% 33%",
       grid_template_areas = '''
       " . . ."
       ". up ."
       "left deselect right"
       "p_frame down n_frame"
       ''')
       )

out = widgets.Output()

select_add = widgets.Button(description="Add Points")
select_modify = widgets.Button(description="Modify Points")
status = widgets.Label(value="foo")

show_points_selector = widgets.ToggleButton(description = "Show Points",value=True)

control_panel = widgets.VBox(children=[arrow_control,select_add,select_modify,show_points_selector ,status])

image_to_view = display_video

all_widgets = widgets.HBox(children=[control_panel, out])
display(all_widgets)

control_state = "Add"
click_to_advance = True

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets
%matplotlib notebook
image_index = 0

with out:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_title('click on points')
    ax.imshow(image_to_view[image_index])
    plot_points = ax.plot([], 'o', picker=5,alpha = 1)  # 5 points tolerance
    global highlight
    highlight, = ax.plot([], [], 'o', color='orange')


selected = None

def onpick(event):
    this_point = event.artist
    x_value = this_point.get_xdata()
    y_value = this_point.get_ydata()
    ind = event.ind
    global selected
    selected = ind
    highlight.set_data(x_value[ind][0],y_value[ind][0])
    fig.canvas.draw_idle()
    plt.gca().set_title("{},{}".format(x_value[ind][0],y_value[ind][0]))
    #plt.gca().set_title("Click at {:.2f}/{:.2f}\non {:s}".format(event.mouseevent.x, event.mouseevent.y, event.artist.__repr__()))
    plt.gcf().canvas.draw_idle() 
       
   
    
def onclick(event):
    global status
    status.value = "click"
    global control_state
    if(control_state == "Add"):
        newx = np.append(ax.lines[0].get_xdata(),event.xdata)
        newy = np.append(ax.lines[0].get_ydata(),event.ydata)
        status.value = "clack"
        ax.lines[0].set_data(newx,newy)
        if click_to_advance:
            global image_index
            image_index = (image_index+1)%len(image_to_view)
            ax.imshow(image_to_view[image_index])
        ax.set_title("Frame {} / {}".format(image_index,len(image_to_view)))
        plt.canvas.draw_idle()
        
click_control = fig.canvas.mpl_connect('button_press_event', onclick)  

def move_up(event):
    a,b = ax.get_ylim()
    delta = 0.005*(b-a)
    highlight.set_data(highlight.get_xdata(),highlight.get_ydata()+delta)
    fig.canvas.draw_idle()
    if selected is not None:
        foo = ax.lines[0].get_ydata()
        mod = np.zeros_like(foo)
        mod[selected] = delta
        ax.lines[0].set_ydata(foo+mod)
    
def move_down(event):
    a,b = ax.get_ylim()
    delta = 0.005*(b-a)
    highlight.set_data(highlight.get_xdata(),highlight.get_ydata()-delta)
    fig.canvas.draw_idle() 
    if selected is not None:
        foo = ax.lines[0].get_ydata()
        mod = np.zeros_like(foo)
        mod[selected] = delta
        ax.lines[0].set_ydata(foo-mod)
    
def move_left(event):
    a,b = ax.get_xlim()
    delta = 0.005*(b-a)
    highlight.set_data(highlight.get_xdata()-delta,highlight.get_ydata())
    fig.canvas.draw_idle()
    if selected is not None:
        foo = ax.lines[0].get_xdata()
        mod = np.zeros_like(foo)
        mod[selected] = delta
        ax.lines[0].set_xdata(foo-mod)
    
def move_right(event):
    a,b = ax.get_xlim()
    delta = 0.005*(b-a)
    highlight.set_data(highlight.get_xdata()+delta,highlight.get_ydata())
    fig.canvas.draw_idle()
    if selected is not None:
        foo = ax.lines[0].get_xdata()
        mod = np.zeros_like(foo)
        mod[selected] = delta
        ax.lines[0].set_xdata(foo+mod)

def remove_selection(event):
    global selected
    selected = None
    highlight.set_data([],[])
    
def set_state(b):
    global control_state
    global pick_control
    if b.description == "Add Points":
        control_state = "Add"
        highlight.set_data([],[])
        fig.canvas.draw_idle()
        try:
            fig.canvas.mpl_disconnect(pick_control);
        except:
            pass
    elif b.description == "Modify Points":
        control_state = "Modify"
        pick_control = fig.canvas.mpl_connect('pick_event', onpick);
    else:
        pass
    global status
    status.value = control_state
    
def set_next_frame(event):
    global image_index
    image_index = (image_index+1)%len(image_to_view)
    ax.imshow(image_to_view[image_index])
    fig.canvas.draw_idle()

def set_prev_frame(event):
    global image_index
    image_index = (image_index-1)%len(image_to_view)
    ax.imshow(image_to_view[image_index])
    fig.canvas.draw_idle()    
    
def show_points_toggle(event):
    status.value = str(event['new'])
    if(event['new']):
        ax.lines[0].set_alpha(1)
    else:
        ax.lines[0].set_alpha(0)
    fig.canvas.draw_idle()
    
up_control.on_click(move_up)
down_control.on_click(move_down)
left_control.on_click(move_left)
right_control.on_click(move_right)
deselect.on_click(remove_selection)
next_frame.on_click(set_next_frame)
prev_frame.on_click(set_prev_frame)
show_points_selector.observe(show_points_toggle, 'value')


select_add.on_click(set_state)
select_modify.on_click(set_state)



----

In [ ]:
Particle = ax.lines[0].get_data()
figg, (axx,ayy) = plt.subplots(2)
axx.plot(np.arange(len(Particle[0])),Particle[0],label = "position",marker='o')
ayy.plot(np.arange(len(Particle[1])),Particle[1],label = "position",marker='o')
velocityx = np.gradient(Particle[0])
axx.plot(np.arange(len(velocityx)),5*velocityx,label = "velocity",marker='o')
velocityy = np.gradient(Particle[1])
ayy.plot(np.arange(len(velocityy)),5*velocityy,label = "velocity",marker='o')
axx.set_title("X coordinate")
ayy.set_title("Y coordinate")
axx.legend()
ayy.legend()

In [ ]:
fig2, (ax2,ay2) = plt.subplots(2)
ax2.hist(velocityx)
ay2.hist(velocityy)
plt.show()

----
----
----

In [ ]:
print(velocityx,'\n\n',velocityy)

In [ ]:
from scipy import optimize
# Generate numpy arrays for the histogram values and bin centers
dxhist = np.histogram(velocityx, bins=10)
errdx = np.sqrt(dxhist[0])
errdx[errdx<1] = 1.4 # This sets the correct Poisson uncertainty when N = 0
cbins = (dxhist[1][:-1]+dxhist[1][1:])/2

# Define a Gaussian fit function and weighted residual
def fitfunc(p,x):
    return p[0]/(p[2]*np.sqrt(2*np.pi))*np.exp(-(x-p[1])**2/(2*p[2]**2))
def residual(p, x, y, err):
    return (fitfunc(p, x)-y)/err

#Provide an initial guess for fit parameters and perform the fit
p0 = [3000., 0., 1.]
pf, cov, info, mesg, success = optimize.leastsq(residual, p0, args=(cbins, dxhist[0], errdx),
                                                full_output=1)

if cov is None:
    print('Fit did not converge')
    print('Success code:', success)
    print(mesg)
else:
    chisq = sum(info['fvec']*info['fvec'])
    dof = len(cbins) - len(pf)
    pferr = [np.sqrt(cov[i,i]) for i in range(len(pf))]
    print('Converged with chi-squared', chisq)
    print('Number of degrees of freedom, dof =', dof)
    print('Reduced chi-squared ', chisq/dof)
    print('Inital guess values:')
    print('  p0 =', p0)
    print('Best fit values:')
    print('  pf =', pf)
    print('Uncertainties in the best fit values:')
    print('  pferr =', pferr)
    
    #Let's save the standard deviation values for use later
    sigma_x = pf[2]
    dsigma_x = pferr[2]
    
    
print()   
dyhist = np.histogram(velocityy, bins=10)
errdy = np.sqrt(dyhist[0])
errdy[errdy<1] = 1
cbins = (dyhist[1][:-1]+dyhist[1][1:])/2

p0 = [3000., 0., 1.]
pfy, cov, info, mesg, success = optimize.leastsq(residual, p0, args=(cbins, dyhist[0], errdy),
                                                full_output=1)

if cov is None:
    print('Fit did not converge')
    print('Success code:', success)
    print(mesg)
else:
    chisq = sum(info['fvec']*info['fvec'])
    dof = len(cbins) - len(pf)
    pferr = [np.sqrt(cov[i,i]) for i in range(len(pf))]
    print('Converged with chi-squared', chisq)
    print('Number of degrees of freedom, dof =', dof)
    print('Reduced chi-squared ', chisq/dof)
    print('Inital guess values:')
    print('  p0 =', p0)
    print('Best fit values:')
    print('  pf =', pf)
    print('Uncertainties in the best fit values:')
    print('  pferr =', pferr)
    
    #Let's save the standard deviation values for use later
    sigma_y = pf[2]
    dsigma_y = pferr[2]    

----

In [ ]:
fig,(ax,ay) = plt.subplots(2)
ax.errorbar(cbins, dxhist[0], errdx, fmt='k.', label='Data')

X = np.linspace(min(cbins), max(cbins), 5000)
ax.plot(X, fitfunc(pf, X), 'r-', label='Fit')

ax.set_title('Horizontal displacement between consecutive frames')
ax.set_xlabel('$\Delta x$ (pixels)')
ax.set_ylabel('Counts')
ax.legend()

textfit1 = '$C(x) = \\frac{N}{\sigma\sqrt{2\pi}}\\exp\\left(-\\frac{(x-x_0)^2}{2\sigma^2}\\right)$ \n' \
               '$N = %.0f \pm %.0f$ \n' \
               '$x_0 = %.3f \pm %.3f$ pixels\n' \
               '$\sigma= %.3f \pm %.3f$ pixels \n' \
               '$\chi^2/N = % .2f$' \
                %(pf[0], pferr[0], pf[1], pferr[1], pf[2], pferr[2], chisq/dof)

ax.text(0.02, .94, textfit1, transform=ax.transAxes, fontsize=12,
             verticalalignment='top');

ay.errorbar(cbins, dyhist[0], errdy, fmt='k.', label='Data')

Y = np.linspace(min(cbins), max(cbins), 5000)
ay.plot(Y, fitfunc(pfy, Y), 'r-', label='Fit')

ay.set_title('Vertical displacement between consecutive frames')
fig.tight_layout()
ay.set_xlabel('$\Delta y$ (pixels)')
ay.set_ylabel('Counts')
ay.legend()

textfit1 = '$C(x) = \\frac{N}{\sigma\sqrt{2\pi}}\\exp\\left(-\\frac{(x-x_0)^2}{2\sigma^2}\\right)$ \n' \
               '$N = %.0f \pm %.0f$ \n' \
               '$x_0 = %.3f \pm %.3f$ pixels\n' \
               '$\sigma= %.3f \pm %.3f$ pixels \n' \
               '$\chi^2/N = % .2f$' \
                %(pf[0], pferr[0], pf[1], pferr[1], pf[2], pferr[2], chisq/dof)

#ax.set_xlim(left=-2.3)
ay.text(0.02, .94, textfit1, transform=ax.transAxes, fontsize=12,
             verticalalignment='top');


In [ ]:
# Particle radius
a = 0.96*10**(-6)/2  # m

# Video frame rate and time between frames
framerate = 10 # frames per second
dt = 1/framerate # s

# Absolute temperature and Boltzmann constant
T = 297 # K
k_B = 1.3806*10**(-23) # m^2 kg/(s^2 K)

# Conversion factor between real length and pixel length
conversion = 0.01*10**(-3)/61.0 # m/pixel

# Viscosity of water (at temperature T)
eta = 8.90*10**(-4)  # Pa s = kg/(s m)

In [ ]:
#Take the average of your two sigma values, and convert to real length units
sigma = (sigma_x + sigma_y)/2 * conversion                 # m
dsigma = np.sqrt(dsigma_x**2 + dsigma_y**2)/2 * conversion # m

#Compute the diffusion coefficient
D = sigma**2/2/dt     # m^2/s
dD = D*(dsigma/sigma) # m^2/s

#Compute the expected diffusion coefficient
D_predicted = k_B*T/(6*np.pi*eta*a)

print('D_predicted = {:0.4f} x 10^(-13) m^2/s'.format((D_predicted/10**(-13))))
print('D_exp = ({:0.4f} +/- {:0.4f}) x 10^(-13) m^2/s'.format(D/10**(-13), dD/10**(-13)))